In [1]:
!pip install sympy

In [2]:
from sympy import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, Math, Latex
init_printing(use_latex='mathjax')

In [3]:
def print_eq(eq, *exprs, rev_int2=False):
    if type(eq) == str:
        eq = Symbol(eq)
    out = '$$ {} ' + '= {}' * len(exprs) + '$$'
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    if rev_int2:
        exprs[-2] += '|_0^x'
    display(Latex(out.format(latex(eq), *exprs)))
    
def get_degree(el):#только для массивов без элементов в виде чисел
    '''
    Эта функция используется для сортировки членов интеграла по степеням,
    а то там порядок сбивается, и отрицательные члены сначала идут
    '''
    if ask(Q.extended_real(el)):
        return el
    arg = el.args
    if len(arg) > 1:
        return get_degree(arg[-1])
    if len(arg) == 0:
        return 1

def s_o(sym, order):#sym
    sym = str(sym)
    if len(sym) > 1:
        tail = sym[1:]
        sym=sym[0]
    else:
        tail = ''
    if order == 0:
        return Symbol(sym + tail)
    if 1 <= order <=3:
        order = '\'' * order
    else:
        order = f'({order})'
    return Symbol(sym + '^{'+order+'}' + tail)

def to_s(i):
    return Symbol(latex(i))

In [4]:
def get_ser(expr, i):
    expr_dif = expr.diff(x, i)#дифференциал
    print('Дифференциал')
    print_eq(Eq(s_o(y, i), expr_dif))#выводим его
    print_eq(Eq(s_o(y_f(x0), i), expr_dif.subs(x, x0)))#выводим чему равен при подстановке
    ser = expr.series(x0=x0, n=i + 1).removeO()#ряд
    return ser

In [5]:
def print_answer(ser, good_detominator, answ_by_count=False):
    integral = integrate(ser, (x))
    integral_args = sorted(integral.args, key=get_degree)# интеграл от ряда
    l1 = integral_args[-1].subs(x, a)# последний член интеграла
    l2 = integral_args[-2].subs(x, a)# предпоследний
    cur_acc = abs(l2 + l1)#считаем здесь их точность
    print('Разложение в ряд подынтегрального выражения')
    print_eq(expr, ser)
    print('Находим неопределённый интеграл')
    if answ_by_count:
        integ_need = integral # нужная часть интеграла для итогового счёта
        integ_2_last = ''
        pp = ''
    else:
        print('Убираем последние 2 члена т.к. они дают меньше нужной точности')
        integ_need = Add(*integral_args[:-2])
        integ_2_last = f'\\bigg[\\bigg({latex(Add(*integral_args[-2:]).subs(x, to_s(a)))}\\bigg) < {accuracy}\\bigg] +'
        pp = latex(integ_need.subs(x, to_s(a))) + r'\approx'
    answer = integ_need.subs(x, a)
    print_eq(latex(Integral(expr, (x, 0, a))) + r'\approx' + latex(Integral(ser, (x, 0, a))), 
             f'\\bigg({latex(integral)}\\bigg)' + f'\\bigg|_0^{{{latex(a)}}}',
             integ_2_last + latex(integ_need.subs(x, to_s(a))) + r'\approx' + pp + str(answer)
            )
#     print_eq(Eq(Add(integ_need), answer)) 
    try:
        print('Численный ответ = ', answer.p / answer.q)
    except AttributeError:
        print('Численный ответ = ', answer)
    try:
        print('Точность %12.10f' % (cur_acc.p / cur_acc.q))
    except AttributeError:
        print('Точность %12.10f' % cur_acc)

In [6]:
def main(expr, find_by_acc = True):
    print('Выражение')
    if good_detominator:
        expr_dif = expr#дифференциал
    else:
        expr_dif = new_expr
    print_eq(expr_dif)
    i = 0
    found_els = 0
    while True:
        print(i, 'порядок')
        ser = get_ser(expr_dif, i)
        integral = sorted(integrate(ser, (x)).args, key=get_degree)
        if Add(*integral).subs(x, a) != 0:
            found_els += 1
        if find_by_acc:
            if found_els >= 2:
                l1 = integral[-1].subs(x, a)# последний член интеграла
                l2 = integral[-2].subs(x, a)# предпоследний
                cur_acc = abs(l2 + l1)#считаем здесь их точность
                if cur_acc < accuracy and found_els >= 4:
                    print_answer(ser, good_detominator, False)
                    break
        else:
            if found_els == n:
                print_answer(ser, good_detominator, True)
                break
        i +=1

In [7]:
cosh.__name__='ch'
sinh.__name__='sh'

In [8]:
x = Symbol('x')
y = Symbol('y')
y_f = Function('y')

# Вставьте здесь ваши параметры

In [9]:
x0 = 0 #в этой задаче он должен быть 0, поэтому не меняем
accuracy = 0.001 # до производной какой степени считать
n = 4 # Надо 4 члена, хоть и в практической написано 3
a = 0.5# a в интеграле(верхняя циферка). Нижняя у всех 0
expr = sin(2*x) / (x + 1)#<------- здесь должно быть ваше выражение
print('Проверьте, что ваше выражение корректно отображается!')
print_eq(Integral(expr, (x, 0, a)))
print('Вычислить с точностью =', accuracy)

Проверьте, что ваше выражение корректно отображается!


<IPython.core.display.Latex object>

Вычислить с точностью = 0.001


In [10]:
good_detominator = True
for el in expr.args:
    degree = get_degree(el)
    if (degree is None or ask(Q.extended_real(el))) or degree > 1:
        continue
    denominator = el
    if not ask(Q.extended_real(denominator.subs(x, 0))) or denominator.subs(x, 0) == 0:
        print('В знаменателе только x, поэтому его мы игнорируем при дифференцировании, а потом, получив дифференциал, поделием его на этот знаменатель')
        print('В коде ниже будет результат вместе с делением')
        new_expr = expr / denominator
        print_eq(new_expr)
        good_detominator = False
        break

# Вычисление по точности

но Виклянчук сказала, что нужно минимум 4 члена высчитать, поэтому минимум 4 члена и выведется

In [11]:
main(expr, True)

Выражение


<IPython.core.display.Latex object>

0 порядок
Дифференциал


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

Разложение в ряд подынтегрального выражения


<IPython.core.display.Latex object>

Находим неопределённый интеграл
Убираем последние 2 члена т.к. они дают меньше нужной точности


<IPython.core.display.Latex object>

Численный ответ =  0.175347222222222
Точность 0.0005983383


# Вычисление по количеству членов

In [12]:
main(expr, False)

Выражение


<IPython.core.display.Latex object>

0 порядок
Дифференциал


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

Разложение в ряд подынтегрального выражения


<IPython.core.display.Latex object>

Находим неопределённый интеграл


<IPython.core.display.Latex object>

Численный ответ =  0.172916666666667
Точность 0.0062500000
